# Tarea 1

Implementar la clase `OisCashflow`.

## Configuración Inicial

In [2]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
from scipy.optimize import root_scalar

# Modificado por AD
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

import pandas as pd
import math
import datetime as dt
import pytz
from pandas import DataFrame
import numpy 
import plotly.express as px

In [3]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [4]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

## `Qcf.time_series`

In [5]:
ts = Qcf.time_series()

fecha1 = Qcf.QCDate(13, 1, 1969)
ts[fecha1] = 19690113

fecha2 = Qcf.QCDate(14, 1, 1969)
ts[fecha2] = 19690114

print(f'ts[fecha1]: {ts[fecha1]}\n')

dl = Qcf.time_series_dates(ts)
for d in dl:
    print(d)
print()
    
vl = Qcf.time_series_values(ts)
for v in vl:
    print(v)

ts[fecha1]: 19690113.0

13-1-1969
14-1-1969

19690113.0
19690114.0


## Clase `OisCashflow`

La clase `OisCashflow` es una [`dataclass`](https://realpython.com/python-data-classes/). La tarea consiste en:

- Implementar los 3 métodos de la clase que están con `pass`.
- Implementar la función `present_value`.
- Implementar la función `set_expected_rate`.

Construcción de un objeto `Qcf.InterestRateIndex`.

In [6]:
# Se importa el archivo Excel a un DataFrame de pandas
Datos=pd.read_excel('../data/SOFR-10012019-10292020.xls','nice format') #buscar data que está en planilla excel. 
Data=Datos.iloc[::-1] #Se da vuelta. 
Data=Data.reset_index(drop=True) #Para que se vea mejor. 
Data

,DATE,BENCHMARK NAME,RATE\n(PERCENT)
0,2019-10-01,SOFR,1.88
1,2019-10-02,SOFR,1.85
2,2019-10-03,SOFR,1.84
3,2019-10-04,SOFR,1.82
4,2019-10-07,SOFR,1.83
...,...,...,...
265,2020-10-22,SOFR,0.07
266,2020-10-23,SOFR,0.08
267,2020-10-26,SOFR,0.09
268,2020-10-27,SOFR,0.09


In [7]:
for i in range(270):
    Data.iloc[i,0] = dt.datetime.strptime(Data.iloc[i,0],'%Y-%m-%d')
    
# Se visualiza el resultado
Data.style.format({'DATE': '{%Y-%m-%d}'})
Data.style.format({'RATE(PERCENT)': '{0:.2%}'})
for i in range(270):
    
     Data.iloc[i,0] = Qcf.QCDate(Data.iloc[i,0].day, Data.iloc[i,0].month, Data.iloc[i,0].year)


In [8]:
#Agregarle Fecha final.  correrlo desde el inicio, no se porque la `primera vez sale error correr desde in 10
Fecha_Final=Data.iloc[1:270,0]
Fecha_Final=Fecha_Final.reset_index(drop=True)
Data["Fecha Final"]=Fecha_Final
Data= Data[['DATE','Fecha Final','RATE\n(PERCENT)']]
#Cambio nombre a las columnas 
Data.columns = ['Fecha inicial', 'Fecha Final', 'Tasa']
Data.iloc[269,1]=Qcf.QCDate(29,10,2020)
Tasa1=Data['Tasa']/100
Data["Tasa Final"]=Tasa1
Data= Data[['Fecha inicial','Fecha Final','Tasa Final']]
Data

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/indexing.py:964: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Fecha inicial,Fecha Final,Tasa Final
0,1-10-2019,2-10-2019,0.0188
1,2-10-2019,3-10-2019,0.0185
2,3-10-2019,4-10-2019,0.0184
3,4-10-2019,7-10-2019,0.0182
4,7-10-2019,8-10-2019,0.0183
...,...,...,...
265,22-10-2020,23-10-2020,0.0007
266,23-10-2020,26-10-2020,0.0008
267,26-10-2020,27-10-2020,0.0009
268,27-10-2020,28-10-2020,0.0009


## Datos 

In [9]:
#ver calendario de dias. 
calendar= get_cal(BusCal.NY)
#Hacer el fixings para poder trabajarlo en la función. 
fixings=Qcf.time_series()
for i in range(270):
    fixings[Data.iloc[i,0]]=Data.iloc[i,2] #a la fecha le asigno una tasa overnight 

In [10]:
vl = Qcf.time_series_dates(fixings)

## Primera parte tarea

In [11]:
@dataclass # syntactic sugar
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional:float
    currency:Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
  
    interest_rate: Qcf.QCInterestRate
       
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float
    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula la tasa equivalente desde `start_date` a `accrual_date`. La tasa equivalente
        se calcula como:
        
        (P - 1) * 360 / (accrual_date - start_date)
        
        donde P es el producto de los factores de capitalización de todas las tasas overnight
        entre `start_date` y `accrual_date`. Los valores de esas tasas deben estar almacenados
        en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        #Llamamos a las variables
        vl = Qcf.time_series_dates(fixings)
        valor = Qcf.time_series_values(fixings)
        j=0
        accrual_date = calendar.next_busy_day(accrual_date) #por si es fin de semana o feriado 
        for i in vl: #saco los indices que quiero 
    
            if i==self.start_date: # el indice inicial
                z=j
            if i==accrual_date:#el indice final 
                k=j
            j=j+1

        #Calculamos las tasas 
        qc_tasa=[]
        for i in range(z,k):
            
            qc_tasa.append(Qcf.QCInterestRate(valor[i], Qcf.QCAct360(), Qcf.QCLinearWf()))
        
        
        #Ahora sacar factor de capitalización 
        Factor=[]
        for i in range(z,k):
            Factor.append(qc_tasa[i-z].wf(vl[i], vl[i+1])) #factor de capitalizacion entre t y t+1
        
        #Ahora hacer producto de los factores de capitalizacion.
        #WF=DataFrame(Factor)
        #p=WF.product() #multiplicacion factores
        p=numpy.product(Factor)
        dias = self.start_date.day_diff(accrual_date) #dias transcurridos 
        
        Tasa_eq=(p-1) * 360/dias #tasa equivalente 
        
        
        return Tasa_eq
    
    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula el interés devengado desde `start_date` a `accrual_date` utilizando la tasa equivalente
        que se calcula con el método anterior.
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        #Se necesita la tasa equivalente.
        #Esta se obtuvó en la función de arriba.
        accrual_date = calendar.next_busy_day(accrual_date)
        tasa=self.get_accrued_rate(accrual_date, fixings) #necesito la tasa equivalente que se calcula en la funcion anterior
        dias = self.start_date.day_diff(accrual_date) #diferencia de dias
        #nocional por la tasa equivalente ^dias/360
        interes=self.notional*tasa*dias/360 #intereses devengados 
        return interes
    
    def amount(self,fixings: Qcf.time_series) -> float:
        """
        Calcula el flujo total al vencimiento (amortización más intereses devengados hasta end_date).
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        #El monto entrega la amortizacion mas intereses. 
        intereses=self.get_accrued_interest(self.end_date, fixings) #necesito los intereses de la funcion anterior
        Monto=self.amortization+intereses
       
        
        return Monto

In [12]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

Construcción de una instancia de `OisCashflow`.

In [13]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(28, 10, 2020),
    Qcf.QCDate(28, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    10000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

In [14]:
#prueba. 
accrual_date=Qcf.QCDate(7, 10, 2019)
tasa=ois.get_accrued_rate(accrual_date,fixings)
print(tasa)


0.01838522014566024


In [15]:
print(ois.get_accrued_interest(accrual_date,fixings))

3064.2033576100407


In [16]:
print(ois.amount(fixings))

10078517.177790914


## Funciones

In [17]:
frmt = {
    'tasa': '{:.6%}',
    'df': '{:.6%}',
    'valor_tasa': '{:.4%}',
    'spread': '{:.4%}',
    'nominal': '{:,.2f}',
    'interes': '{:,.2f}',
    'amortizacion': '{:,.2f}',
    'flujo': '{:,.2f}',
}

In [18]:
#Importar curva a utilizar. 
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
df_curva.head().style.format(frmt)

,plazo,tasa,df
0,1,0.081111%,99.999778%
1,7,0.084051%,99.998388%
2,14,0.077967%,99.997010%
3,21,0.077358%,99.995549%
4,33,0.078067%,99.992942%


In [19]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct360(),Qcf.QCLinearWf(), df_curva)

In [20]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series) -> None:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Ver por ejemplo los casos cuando usamos el objeto Qcf.ForwardRates() en
    el notebook 9.
    """
    val = Qcf.time_series_values(fixings)
    dates = Qcf.time_series_dates(fixings)
    if val_date < ois_cashflow.start_date: # este sería el caso en que la fecha esta antes de los fixings, no conozco nada, 
        #por lo tanto se ocupa siempre la curva cero cupón. 
        
        #defino los plazos 
        t1 = val_date.day_diff(ois_cashflow.start_date) #a corto plazo
        t2 = val_date.day_diff(ois_cashflow.end_date) #a largo plazo.
        
        
        #Sacar factores de descuento. 
        df_1=zcc.get_discount_factor_at(t1) #factor de descuento a corto plazo
        df_2=zcc.get_discount_factor_at(t2) #factor de descuento a largo plazo. 
        #esto es solo para saber si entraba en el condicional que corresponde, por eso queda comentado. 
        #f=1
        #print(f)
        #division 
        IP=df_1/df_2
        tasa_esperada=(IP-1)*360/(t2-t1) #tasa esperada para cashflow
        ois_cashflow.interest_rate.set_value(tasa_esperada) #actualizo el valor en cashflow 
    else: #este es el caso en el que ya conozco ciertos fixings, y los que no conozco uso la curva cero cupon
        #para proyectar a futuro. 
        #Ver que fixings tengo hasta el momento. 
        #lo que se quiere ver en este caso es cuando t1<t0<t2. Entonces necesito los factores desde t1 a t0 
        #y también necesito los factores desde t0 a t2. 
        t1 = val_date.day_diff(ois_cashflow.start_date)
        t2 = val_date.day_diff(ois_cashflow.end_date)
        #Llamo a los valores que tengo de fixings 
        vl = Qcf.time_series_dates(fixings)
        valor = Qcf.time_series_values(fixings)
        j=0
        for i in vl: #saco los indices que quiero 
    
            if i==ois_cashflow.start_date: # el indice inicial
                z=j
            if i==val_date:#el indice final 
                k=j
            j=j+1

        #Calculamos las tasas 
        qc_tasa=[]
        for i in range(z,k):
            
            qc_tasa.append(Qcf.QCInterestRate(valor[i], Qcf.QCAct360(), Qcf.QCLinearWf()))
        
        
        #Ahora sacar factor de capitalización 
        Factor=[]
        for i in range(z,k):
            Factor.append(qc_tasa[i-z].wf(vl[i], vl[i+1])) #factor de capitalizacion entre t y t+1
        
        #Ahora hacer producto de los factores de capitalizacion.
        wf_1=numpy.product(Factor)
        #Saco el factor de descuento que no conozco con la curva zcc 
        df_2=zcc.get_discount_factor_at(t2)
        #el inverso es el factor de capitalizacion. 
        wf_2=1/df_2
        #multiplico los factores y saco la tasa. 
        tasa_esperada=(wf_1*wf_2-1)*360/(t2-t1)
        ois_cashflow.interest_rate.set_value(tasa_esperada)
    
    return tasa_esperada

In [21]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(28, 10, 2020),
    Qcf.QCDate(28, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    10000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

In [22]:
#prueba 
val_date=Qcf.QCDate(7, 7, 2019)
tasa=set_expected_rate(val_date,ois,zcc,fixings)
print(ois.interest_rate)


0.000633 Act360 Lin


In [23]:
def present_value(val_date: Qcf.QCDate, ois_cashflow: OisCashflow, zcc: Qcf.ZeroCouponCurve) -> float:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    """
    #el valor presente desde val date hasta end date me debería dar el nominal. 
    #es decir quiero llegar a diez millones en este caso.
    #debo descontar el monto con el factor de descuento. 
    #esto sería para el caso t0<t1<t2 
    plazo = val_date.day_diff(ois_cashflow.end_date)#entre la fecha hoy y la de vencimiento. 
    df=1/(1+ois_cashflow.interest_rate.get_value()*(plazo/360))
    #la tasa que se quiere usar es la que pasó por la función anterior y cambió el estado del cashflow
    intereses=ois_cashflow.notional*ois_cashflow.interest_rate.get_value()*plazo/360
    #esta ecuacion se vió en clases, pero se cambió el valor de la tasa, ya que está tiene que pasar por la funcion anterior.  
    valor_presente=(ois_cashflow.amortization+intereses)*df
    return valor_presente

In [24]:
valor_presente=present_value(val_date, ois, zcc)
print(f'Valor Presente: {valor_presente:,.2f}')

Valor Presente: 10,000,000.00


## Tests

In [25]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [26]:
amount_tol = 10000
rate_tol = .001

In [27]:
exitos = 0

In [28]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [29]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

### Objeto `fixings`

In [30]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [31]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [33]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [34]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [35]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

accrued rate: 1.04766527%


Se verificará el cálculo usando `df_fixings`.

In [36]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [37]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [38]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [39]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [40]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [41]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [42]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [43]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [44]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [45]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [46]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [47]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

La diferencia es: 0.03775213%


In [48]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.accrued_interest`

In [49]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

accrued interest: 75,082.68


In [50]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

La diferencia es: 0.000000


In [51]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.amount`

In [52]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

amount: 1,077,850.90


In [53]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [54]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

La diferencia es: 2823.854185


In [55]:
suma_exito(amount, check_amount, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [56]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [57]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [58]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [59]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [61]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.06121277%
Check tasa esperada es: 0.06121277%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [62]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.06924275%
Check tasa esperada es: 0.06924275%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [63]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.22166384%
Check tasa esperada es: 0.22166384%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [64]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.76574655%
Check tasa esperada es: 0.76574655%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [65]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.76618836%
Check tasa esperada es: 0.76574655%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `present_value`

In [67]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 1,007,461.7659
Amount: 1,077,850.8996
Df: 99.911652%
VP check es: 1,076,898.6424
Suma medio punto por obtener resultado.


In [68]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 1,006,331.2547
Amount: 1,077,850.8996
Df: 99.929653%
VP check es: 1,077,092.6609
Suma medio punto por obtener resultado.


In [69]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 1,018,526.1327
Amount: 1,077,850.8996
Df: 99.935019%
VP check es: 1,077,150.5062
Suma medio punto por obtener resultado.


In [70]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 1,000,000.0000
Amount: 1,077,850.8996
Df: 0.000000%
VP check es: 0.0000
Suma medio punto por obtener resultado.


In [71]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 997,317.5415
Amount: 1,077,850.8996
Df: 0.000000%
VP check es: 0.0000
Suma medio punto por obtener resultado.


In [72]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 14.5
